In [1]:
import os
print(os.getcwd())
os.chdir('/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1')
print(os.getcwd())

/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1/tests
/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1


# Multihead-GRAG 随机网络聚类测试

In [2]:
from src.gnn_clustering.data_loader import load_random_data
from src.gnn_clustering.utils import get_device
from tests.utils import test_model_performance

# 加载随机数据并设置设备
device = get_device()
data = load_random_data(num_nodes=1024, num_edges=1500).to(device)

# 执行测试
report_df = test_model_performance(data=data, device=device, num_tests=3)

/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1/src/gnn_clustering/data_loader.py:93: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  edge_index = torch.tensor([source_indices, target_indices], dtype=torch.long)


Epoch 1, Loss: 715.1267
Epoch 20, Loss: 5.4267
Epoch 40, Loss: 1.9196
Epoch 60, Loss: 1.2178
Epoch 80, Loss: 1.0255
Epoch 100, Loss: 0.9518
Epoch 120, Loss: 0.9101
Epoch 140, Loss: 0.8841
Epoch 160, Loss: 0.8664
Epoch 180, Loss: 0.8527
Epoch 200, Loss: 0.8424
Epoch 1, Loss: 695.1320
Epoch 20, Loss: 9.0670
Epoch 40, Loss: 1.6358
Epoch 60, Loss: 1.1167
Epoch 80, Loss: 0.9872
Epoch 100, Loss: 0.9315
Epoch 120, Loss: 0.9017
Epoch 140, Loss: 0.8834
Epoch 160, Loss: 0.8711
Epoch 180, Loss: 0.8619
Epoch 200, Loss: 0.8543
Epoch 1, Loss: 661.7258
Epoch 20, Loss: 9.2410
Epoch 40, Loss: 1.9391
Epoch 60, Loss: 1.1657
Epoch 80, Loss: 1.0128
Epoch 100, Loss: 0.9503
Epoch 120, Loss: 0.9163
Epoch 140, Loss: 0.8941
Epoch 160, Loss: 0.8780
Epoch 180, Loss: 0.8655
Epoch 200, Loss: 0.8553


In [3]:
import pandas as pd

# 设置最大显示的列数为 None，表示不限制
pd.set_option('display.max_columns', None)
report_df

,id,nodes,edges,initial_modularity,initial_clusters,initial_modularity_embedding,initial_clusters_embedding,avg_modularity_after_training,avg_clusters_after_training,avg_ami,improvement_leiden (%),improvement_embedding_leiden (%),head_1_modularity,head_1_num_clusters,head_2_modularity,head_2_num_clusters,head_3_modularity,head_3_num_clusters,head_1_ami_with_head_2,head_1_ami_with_head_3,head_2_ami_with_head_3
0,1,1024,3000,0.654268,73,0.837597,31,0.896697,35.333333,0.163023,137.053540,107.055859,0.905513,37,0.887279,35,0.897299,34,0.173098,0.165931,0.150042
1,2,1024,3000,0.658433,73,0.840805,31,0.895627,36.000000,0.174284,136.024017,106.520283,0.892756,37,0.903874,38,0.890253,33,0.184300,0.178144,0.160407
2,3,1024,3000,0.662276,71,0.828889,28,0.898761,34.000000,0.165837,135.707969,108.429615,0.904525,33,0.891377,33,0.900382,36,0.156288,0.170151,0.171071
